In [1]:
import os
os.chdir("../")
!pwd

/home/aditya/MLOPS/E2E-Data-Science-Project


In [7]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/adityaav80/wine_quality.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "ADITYAAV80" 
os.environ["MLFLOW_TRACKING_PASSWORD"] = "7c7d5f1b5994f5230784a07f1f9573168c40c053"

In [8]:
from pathlib import Path
from pydantic import BaseModel

"""
model_evaluation:
    root_dir: artifacts/model_evaluation
    test_data_path: artifacts/data_transformation/test.csv
    model_path: artifacts/model_trainer/model.joblib
    metric_file_name: artifacts/model_evaluation/metrics.json
"""
class ModelEvaluationConfig(BaseModel):
    ## from config
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: str
    all_params:dict
    target_column: str
    mlflow_uri: str

In [ ]:
from src.DataScienceWorkflow.constants import *
from src.DataScienceWorkflow.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self)->ModelEvaluationConfig:

        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        data_model_evaluation_config = ModelEvaluationConfig(
            
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            all_params=params,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/adityaav80/wine_quality.mlflow"
        )

        return data_model_evaluation_config

In [12]:
cm = ConfigurationManager()
data_model_evaluation_config = cm.get_model_evaluation_config()

[2025-03-29 22:03:44,506: INFO: common : Yaml File: config/config.yaml loaded successfully]
[2025-03-29 22:03:44,508: INFO: common : Yaml File: params.yaml loaded successfully]
[2025-03-29 22:03:44,509: INFO: common : Yaml File: schema.yaml loaded successfully]
[2025-03-29 22:03:44,509: INFO: common : created directory at: artifacts]
[2025-03-29 22:03:44,510: INFO: common : created directory at: artifacts/model_evaluation]


In [14]:
import pandas as pd
import joblib
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import numpy as np
import mlflow
from urllib.parse import urlparse
from src.DataScienceWorkflow.utils.common import save_json


class ModelEvaluate:
    def __init__(self,config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self,actual,pred):

        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)

        return rmse,mae,r2_score

    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        x_test = test_data.drop([self.config.target_column],axis = 1)
        y_test = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            y_pred = model.predict(x_test)
            (rmse, mae, r2) = self.eval_metrics(y_test, y_pred)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
    

In [15]:
me = ModelEvaluate(data_model_evaluation_config)
me.log_into_mlflow()

🏃 View run luminous-zebra-957 at: https://dagshub.com/adityaav80/wine_quality.mlflow/#/experiments/0/runs/82159c1b84b54e2c893134d6a8a0eaa5
🧪 View experiment at: https://dagshub.com/adityaav80/wine_quality.mlflow/#/experiments/0
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/aditya/miniconda3/envs/E2E-Data-Sciene-Project/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3549, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_440119/1349868686.py", line 2, in <module>
    me.log_into_mlflow()
  File "/tmp/ipykernel_440119/1107931009.py", line 41, in log_into_mlflow
    save_json(path=Path(self.config.metric_file_name), data=scores)
  File "/home/aditya/miniconda3/envs/E2E-Data-Sciene-Project/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py", line 39, in wrapper_function
    return wrapper(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/aditya/miniconda3/envs/E2E-Data-Sciene-Project/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py", line 136, in __call__
    res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^